In [6]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import time

from libworm.model.beta_neuron import NeuronNetwork, from_connectome
from libworm.data import connectomes, traces
from libworm import preprocess
from libworm.functions import set_neurons, tcalc_s_inf, set_trace

In [7]:
_, trace, trace_labels, label2index, timestamps = traces.load_trace()
timestamps = timestamps - timestamps[0]

chemical, gapjn = connectomes.load_cook_connectome()
neurons = connectomes.get_main_neurons(chemical, gapjn)
neurons.sort(key=lambda item: f"AAA{label2index[item]:04d}{item}" if item in label2index else item)
model = from_connectome(chemical, gapjn, neurons)

cell = "SMBVR"

first_removal = [label2index[key] for key in label2index if key not in neurons]
trace = np.delete(trace, first_removal, axis=0)

del_index = 0
size = trace.shape[0]

for i in range(size):
    if i not in label2index.values():
        trace = np.delete(trace, (del_index), axis=0)
    else:
        del_index += 1

trace_pairs = [(time, trace[:, i]) for i, time in enumerate(timestamps)]

In [8]:
def eval(model, start_index, data):
    np.seterr(all='raise')

    try:
        for i in range(15):
            runtime =  data[start_index + i + 1][0] - data[start_index + i][0]
            points = data[start_index + i][1]
            model.big_V[:len(points)] = points
            model.simple_run(0.01, runtime, show_progress=False)
    
        start_index += 15
        error = 0
        
        for i in range(15):
            runtime =  data[start_index + i + 1][0] - data[start_index + i][0]
            points = data[start_index + i][1]
    
            model.simple_run(0.01, runtime, show_progress=False)
            error += np.sum(np.square(model.big_V[:len(points)] - points))
            
    except FloatingPointError:
        error = 10 ** 10

    return error


In [9]:
def reduce2index(array_like):
    return [i for i, item in enumerate(array_like.flatten()) if item != 0]

def reduce(array_like):
    return [item for item in array_like.flatten() if item != 0]

def expand(values, indices, shape):
    size = 1

    for item in shape:
        size *= item

    items = [0] * size

    for i, index in enumerate(indices):
        items[index] = values[i]

    return np.array(items).reshape(shape)

In [10]:
class ModelFactory:
    def __init__(self, length, G_syn_index, G_gap_index):
        self.length = length
        self.G_syn_index = G_syn_index
        self.G_gap_index = G_gap_index

    def get_spec(self):
        return self.length, len(self.G_syn_index), len(self.G_gap_index)

    def build(self, inital_V_value, short_G_syn, short_G_gap, short_E_syn, G_leak, E_leak):

        big_V = np.array([inital_V_value] * self.length)
        
        G_syn = expand(short_G_syn, self.G_syn_index, (self.length, self.length))
        E_syn = expand(short_E_syn, self.G_syn_index, (self.length, self.length))
        
        G_gapjn = expand(short_G_gap, self.G_gap_index, (self.length, self.length))
        
        return NeuronNetwork(big_V, G_syn, G_gapjn, E_syn, labels=neurons, G_leak = G_leak, E_leak = E_leak)

    def build_random(self, rng):
        neuron_size, syn_size, gap_size = self.get_spec()

        inital_v = rng.uniform(-10, 10)
        G_syn = rng.uniform(0, 100, syn_size)
        E_syn = rng.uniform(-100, 100, syn_size)
        G_gap = rng.uniform(0, 100, gap_size)
        G_leak = rng.uniform(0, 100, neuron_size)
        E_leak = rng.uniform(-100, 100, neuron_size)

        return self.build(inital_v, G_syn, G_gap, E_syn, G_leak, E_leak)

In [13]:
%%timeit
factory = ModelFactory(len(neurons),
                       reduce2index(model.big_G_syn),
                       reduce2index(model.big_G_gap))

neuron_size, syn_size, gap_size = factory.get_spec()
rng = np.random.default_rng(3471)

models = [factory.build_random(rng) for i in range(10)]
evals = [eval(model, 50, trace_pairs) for model in models]

32.7 s ± 22.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
evals

[178600.0650367371,
 269691.58061388356,
 160663.424875712,
 215622.88413042168,
 287234.7054161251,
 310470.172257576,
 228134.74262299977,
 177582.41762163365,
 156940.80237352062,
 265032.6256356979,
 223603.89747492856,
 170624.06261818775,
 210735.18700456113,
 196673.02424813347,
 226197.48743809064,
 213800.12707146388,
 169663.6203905323,
 202950.56829198034,
 170892.86228037614,
 432819.76582474646,
 192680.3572468018,
 229783.63828708057,
 274388.40924473153,
 316808.0184194307,
 232886.46809778258,
 279943.7885531138,
 222205.17010862156,
 290965.4899304097,
 216296.08405867926,
 178536.00336362192,
 226966.4291302451,
 166006.49447164376,
 169706.3627466049,
 174623.32969180332,
 337926.85822736105,
 220434.8246688342,
 169254.94628275567,
 267285.5879765947,
 202868.64589502275,
 302119.6970442218,
 286827.26009175385,
 143533.58474739973,
 167791.04532587726,
 216443.830563712,
 200877.3055651274,
 261134.25151340337,
 184506.99192387846,
 241510.9092803942,
 269291.89818

In [26]:
eval(model, 50, trace_pairs)

######################################################################################################################################################################################################################################################################################################################

53823.49032367453

In [27]:
new_model.report()

Neurons 280 (280)
V_max = -0.12468994397378985 (0)
V_min = -0.12468994397378985 (0)


In [28]:
eval(new_model, 50, trace_pairs)

##

10000000000